In [28]:
import asyncio
import datetime
import os
import time
from pathlib import Path
from typing import Any, Dict, List, Optional

import openai
import tiktoken
from dotenv import load_dotenv
from langchain import OpenAI, PromptTemplate
from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import (
    ConversationChain,
    LLMChain,
    RetrievalQA,
    SequentialChain,
    ConversationalRetrievalChain,
)
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import PythonCodeTextSplitter
from langchain.document_loaders import BSHTMLLoader, PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.memory import (
    ChatMessageHistory,
    CombinedMemory,
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
    ConversationSummaryMemory,
    SimpleMemory,
)

from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, LLMResult, SystemMessage
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
)
from langchain.vectorstores import FAISS, Chroma, Pinecone
from pydantic import BaseModel, validator
import pinecone

from parameters import (
    ACCURACY_TEMPERATURE_MAP,
    HISTORY_MAX_LENGTH,
    HISTORY_MAX_TEXT,
    MODEL,
    system_prompts,
)


load_dotenv()
load_dotenv(".env.local")
openai.api_key = os.getenv("OPENAI_API_KEY")


In [29]:
def create_chat(
    accuracy: str = "medium",
    stream: bool = True,
    model: str = "gpt-3.5-turbo",  # Replace with your model
    session_id: Optional[str] = None,
) -> str:
    chat = ChatOpenAI(
        model_name=model,
        temperature=ACCURACY_TEMPERATURE_MAP[accuracy],
        streaming=stream,
        # callbacks=[MyCustomAsyncHandler()],
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    return chat




### Vector store related

In [50]:
def convert_python_code_base_to_split_texts(folder_path: Path, filetype: str = ".py"):

    if not folder_path.is_dir():
        raise ValueError("folder_path must be a directory")
    documents = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for file in filenames:
            if file.endswith(filetype) and "/.venv/" not in dirpath:
                try:
                    loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                    documents.extend(loader.load_and_split())
                    # documents.extend(loader.load())
                except Exception as e:
                    pass
    print(len(documents))
    python_splitter = PythonCodeTextSplitter(chunk_size=500, chunk_overlap=0)     
    texts = python_splitter.split_documents(documents)
    print(len(texts))
    return texts

def create_pinecone_index(texts, embeddings, index_name):
    pinecone.init(
        api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
        environment=os.getenv("PINECONE_ENV"),  # next to api key in console
    )
    active_indexes = pinecone.list_indexes()
    if index_name not in active_indexes:
        pinecone.create_index(
            index_name,
            dimension=1536,
            metric="cosine",
            pods=1,
            replicas=1,
            pod_type="Starter",
        )
    db = Pinecone.from_documents(texts, embeddings, index_name=index_name)
    return db

def create_chroma_collection(texts, embedding_function, collection_name="example", persist_directory="resources\chroma"):
    # check if the directory exists
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory)
    
    db = Chroma.from_documents(
        documents=texts,
        embedding=embedding_function,
        persist_directory=persist_directory,
        collection_name=collection_name,
    )
    
    return db




In [55]:
collection_name = "wikipedia"
chromadb_persist_directory = 'resources\chromadb'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
local_embedding_function = SentenceTransformerEmbeddings(model_name=model_name, model_kwargs=model_kwargs)



# adding document to the db
loader = TextLoader("resources\example.txt", encoding="utf8")
# loader = PyPDFLoader("resources\gpt4_explain.pdf")
# loader = BSHTMLLoader("resources\gpt4_explain.html", open_encoding="utf8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# adding codebase to the db
texts = convert_python_code_base_to_split_texts(
    Path("resources\langchain"), filetype=".py"
)

collection_name = "langchain_codebase"
db = create_chroma_collection(texts, embedding_function=local_embedding_function, collection_name=collection_name, persist_directory=chromadb_persist_directory)

# db = Chroma.from_documents(
#     documents=texts,
#     embedding=local_embedding_function,
#     persist_directory="resources\chroma",
#     collection_name="fffff",
# )
    


1386
9336


In [58]:
print(db._collection)

db.persist()
db = None
db = Chroma(persist_directory=chromadb_persist_directory, embedding_function=local_embedding_function, collection_name=collection_name)

print(db._collection)

name='langchain_codebase' id=UUID('4eab5dc8-8b2a-422b-9e91-fb24a92bb1b4') metadata=None
name='langchain_codebase' id=UUID('4eab5dc8-8b2a-422b-9e91-fb24a92bb1b4') metadata=None


In [63]:
print(db._collection)
print(db.similarity_search("How to stop streaming in ChatOpenAI?", k=2))


name='langchain_codebase' id=UUID('4eab5dc8-8b2a-422b-9e91-fb24a92bb1b4') metadata=None
[Document(page_content='test_chat_openai_streaming() -> None:\n    """Test that streaming correctly invokes on_llm_new_token callback."""\n    callback_handler = FakeCallbackHandler()\n    callback_manager = CallbackManager([callback_handler])\n    chat = ChatOpenAI(\n        max_tokens=10,\n        streaming=True,\n        temperature=0,\n        callback_manager=callback_manager,\n        verbose=True,\n    )\n    message = HumanMessage(content="Hello")\n    response = chat([message])\n    assert callback_handler.llm_streams > 0', metadata={'source': 'resources\\langchain\\tests\\integration_tests\\chat_models\\test_openai.py'}), Document(page_content='test_promptlayer_chat_openai_streaming() -> None:\n    """Test that streaming correctly invokes on_llm_new_token callback."""\n    callback_handler = FakeCallbackHandler()\n    callback_manager = CallbackManager([callback_handler])\n    chat = Promp

In [74]:
chat = create_chat(model="gpt-3.5-turbo")

# to load a vector store
loaded_db = Chroma(persist_directory=chromadb_persist_directory, embedding_function=local_embedding_function, collection_name=collection_name)

# loaded_db.persist()
# print(loaded_db.similarity_search("who discovered the mode?", k=2))

# actually use it in a chain as data source
# https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html
# chat over documents
memory = ConversationBufferMemory(output_key='answer', memory_key="chat_history", return_messages=True)

chain = ConversationalRetrievalChain.from_llm(
    llm=chat,
    retriever=loaded_db.as_retriever(k=3),
    return_source_documents=True,
    memory = memory,
    verbose=True,
)




In [75]:
response = chain(
    {
        "question": "How to stop callback in ChatOpenAI?",
    }
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
_create_retry_decorator(llm: ChatOpenAI) -> Callable[[Any], Any]:
    import openai

test_openai_chat_streaming_callback() -> None:
    """Test that streaming correctly invokes on_llm_new_token callback."""
    callback_handler = FakeCallbackHandler()
    callback_manager = CallbackManager([callback_handler])
    llm = OpenAIChat(
        max_tokens=10,
        streaming=True,
        temperature=0,
        callback_manager=callback_manager,
        verbose=True,
    )
    llm("Write me a sentence with 100 words.")
    assert callback_handler.llm_streams != 0

test_chat_openai_streaming() -> None:
    """Test that streaming correctly invokes on_llm_new_token callback."""
    callback_handler = FakeCal

In [66]:
print(response["answer"])

print(response["source_documents"][0])

To stop streaming in ChatOpenAI, you can simply set the `streaming` parameter to `False` when you create the `ChatOpenAI` object. For example:

```
chat = ChatOpenAI(
    max_tokens=10,
    streaming=False,  # set streaming to False to stop it
    temperature=0,
    callback_manager=callback_manager,
    verbose=True,
)
```

This will stop the streaming behavior and return the full response at once instead of returning partial responses as they are generated.
page_content='test_chat_openai_streaming() -> None:\n    """Test that streaming correctly invokes on_llm_new_token callback."""\n    callback_handler = FakeCallbackHandler()\n    callback_manager = CallbackManager([callback_handler])\n    chat = ChatOpenAI(\n        max_tokens=10,\n        streaming=True,\n        temperature=0,\n        callback_manager=callback_manager,\n        verbose=True,\n    )\n    message = HumanMessage(content="Hello")\n    response = chat([message])\n    assert callback_handler.llm_streams > 0' metadata

In [76]:
response = chain(
    {
        "question": "Can you write a short example?",
    }
)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How to stop callback in ChatOpenAI?
Assistant: Based on the given context, it seems that you can stop callbacks in ChatOpenAI by passing a "stop" parameter to the "params" dictionary. However, the exact implementation may depend on the specific use case and the structure of the codebase.
Follow Up Input: Can you write a short example?
Standalone question:
Could you provide a brief example of how to stop callbacks in ChatOpenAI by passing a "stop" parameter to the "params" dictionary?
> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an

In [79]:
print(response["answer"])

print(response["source_documents"][0])

print(response.keys())

Yes, to stop callbacks in ChatOpenAI, you can pass a "stop" parameter to the "params" dictionary. Here's an example:

```
from chat_openai import OpenAIChat

# initialize the OpenAIChat object
chat = OpenAIChat(model_name="davinci")

# define the prompts and stop criteria
prompts = ["Hello, how are you today?"]
stop = ["Thanks for chatting, bye!"]

# get the chat parameters
messages, params = chat._get_chat_params(prompts=prompts, stop=stop)

# call the API and pass the stop parameter to the params dictionary
response = chat._call(prompt=prompts[0], run_manager=None, stop=stop, params=params)

# print the API response
print(response)
```

In this example, we first initialize an OpenAIChat object and define the prompts and stop criteria. We then get the chat parameters using the `_get_chat_params` method, passing the prompts and stop criteria as arguments. Finally, we call the API using the `_call` method, passing the prompt, run_manager, stop criteria, and params dictionary as argument